In [24]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import norm
from scipy.optimize import brentq, newton

In [25]:
ticker = 'SPY'
data = yf.download(ticker, start='2022-01-01', end='2025-10-15', progress=False)['Close']
S0 = float(data.iloc[-1])
r = 0.0411
T = 330 / 365  # años a vencimiento
K = 700

# Vol histórica (para seed)
daily_returns = data.pct_change().dropna()
sigma_hist = float(daily_returns.std() * np.sqrt(252))

C:\Users\samue\AppData\Local\Temp\ipykernel_19936\3275334247.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2022-01-01', end='2025-10-15', progress=False)['Close']
C:\Users\samue\AppData\Local\Temp\ipykernel_19936\3275334247.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  S0 = float(data.iloc[-1])
C:\Users\samue\AppData\Local\Temp\ipykernel_19936\3275334247.py:10: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  sigma_hist = float(daily_returns.std() * np.sqrt(252))


In [26]:
# Black-Scholes
def bs_price(S, K, r, T, sigma, option='call'):
    if sigma <= 0 or T <= 0:
        # Límite: precio intrínseco descontado
        if option == "call":
            return max(0.0, S - K*np.exp(-r*T))
        else:
            return max(0.0, K*np.exp(-r*T) - S)
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    if option.lower() == "call":
        return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    else:
        return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)

In [27]:
def implied_vol(price_mkt, S, K, r, T, option="call", brent_bounds=(1e-6, 5.0), tol=1e-8, maxiter=100):
    # Verificación de límites teóricos: el precio debe estar entre [intrínseco, cota superior]
    if option == "call":
        lower = max(0.0, S - K*np.exp(-r*T))
        upper = S
    else:
        lower = max(0.0, K*np.exp(-r*T) - S)
        upper = K*np.exp(-r*T)
        
    if not (lower - 1e-12 <= price_mkt <= upper + 1e-12):
        raise ValueError(f"Precio de mercado fuera de rango teórico [{lower:.4f}, {upper:.4f}]")

    # Función objetivo
    def f(sig):
        return bs_price(S, K, r, T, sig, option) - price_mkt

    a, b = brent_bounds
    fa, fb = f(a), f(b)

    # Ajusta el extremo superior si no cambia de signo
    if fa * fb > 0:
        # intenta expandir b
        for b_try in [10.0, 15.0, 25.0, 50.0]:
            fb = f(b_try)
            if fa * fb <= 0:
                b = b_try
                break

    # Intenta método de Brent
    try:
        iv = brentq(f, a, b, xtol=tol, maxiter=maxiter)
        return iv
    except Exception:
        pass

    # Si Brent falla, intenta método de Newton
    seed = min(max(sigma_hist if 'sigma_hist' in globals() and np.isfinite(sigma_hist) else 0.2, 1e-3), 2.0)
    try:
        iv = newton(lambda s: f(s), seed, fprime=lambda s: bs_vega(S, K, r, T, s), tol=tol, maxiter=maxiter)
        if iv > 0:
            return iv
    except Exception:
        pass

    raise RuntimeError("No se pudo encontrar una volatilidad implícita con los métodos intentados.")

In [28]:
# Ejemplo de uso
C_market = 49.18
iv_call_real = implied_vol(C_market, S0, K, r, T, option="call")

iv_call_real

0.2176568166180272

In [29]:
P_mkt = 14.10  # ejemplo
iv_put_real = implied_vol(P_mkt, S0, K, r, T, option="put")

iv_put_real

0.02420633135773667

## **Funciones de Griegas**

In [30]:
def bs_delta(S, K, r, T, sigma, option='call'):
    if sigma <= 0 or T <= 0:
        if option.lower() == "call":
            return 1.0 if S > K * np.exp(-r*T) else 0.0
        else:
            return -1.0 if S < K * np.exp(-r*T) else 0.0
        
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))

    if option.lower() == "call":
        return norm.cdf(d1)
    else:
        return norm.cdf(d1) - 1    # Sensibilidad del precio respecto al precio del activo subyacente

In [31]:
def bs_theta(S, K, r, T, sigma, option='call'):
    if sigma <= 0 or T <= 0:
        return 0.0
    
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    first_term = - (S * norm.pdf(d1) * sigma) / (2 * np.sqrt(T))

    if option.lower() == "call":
        second_term = r * K * np.exp(-r*T) * norm.cdf(d2)
        return first_term - second_term
    
    else:
        second_term = r * K * np.exp(-r*T) * norm.cdf(-d2)
        return first_term + second_term  # Sensibilidad del precio respecto al tiempo

In [32]:
def bs_gamma(S, K, r, T, sigma):
    if sigma <= 0 or T <= 0:
        return 0.0
    
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    
    return norm.pdf(d1) / (S * sigma * np.sqrt(T))  # Sensibilidad de delta respecto al precio del activo subyacente

In [33]:
def bs_vega(S, K, r, T, sigma):
    if sigma <= 0 or T <= 0:
        return 0.0
    
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    
    return S * norm.pdf(d1) * np.sqrt(T)   # sensibilidad del precio respecto a sigma

In [34]:
def bs_rho(S, K, r, T, sigma, option='call'):
    if sigma <= 0 or T <= 0:
        return 0.0
    
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    if option.lower() == "call":
        return K * T * np.exp(-r*T) * norm.cdf(d2)
    else:
        return -K * T * np.exp(-r*T) * norm.cdf(-d2)  # Sensibilidad del precio respecto a la tasa libre de riesgo